<a href="https://colab.research.google.com/github/ruhul256/PUST-Database-A-New-Bangal-Hand-Digit-Recognition/blob/main/Hand_crafted_feature_extraction_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
# Change directory to the folder containing your dataset
os.chdir('/content/drive/My Drive/Colab Notebooks/main_data')

Mounted at /content/drive


Local Binary Pattern (LBP)

In [ ]:
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from PIL import Image

In [ ]:
X = []  # to store the image data
y = []  # to store the corresponding labels

for digit in range(10):
    # loop over each image file in the folder
    for filename in os.listdir(f'/content/drive/MyDrive/Colab Notebooks/main_data/{digit}/'):
        # load the image file
        img = Image.open(f'/content/drive/MyDrive/Colab Notebooks/main_data/{digit}/{filename}')
        # convert the image to grayscale and then to a numpy array
        img_arr = np.array(img.convert('L'))
        # append the image data and corresponding label to the lists
        X.append(img_arr)
        y.append(digit)

# convert the image data and labels into numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# set LBP parameters
radius = 1
n_points = 8 * radius
# apply LBP to each image in the dataset
X_lbp = []
for img in X:
    lbp = local_binary_pattern(img, n_points, radius)
    X_lbp.append(lbp)

# convert the LBP features into a numpy array
X_lbp = np.array(X_lbp)

In [ ]:
# reshape X_lbp to a 2D array
n_samples, height, width = X_lbp.shape
X_lbp_2d = X_lbp.reshape((n_samples, height*width))
# import PCA from sklearn
from sklearn.decomposition import PCA

# create PCA object with 95% variation
pca = PCA(n_components=0.95)

# fit and transform the LBP features
X_pca_lbp = pca.fit_transform(X_lbp_2d)

In [ ]:
print(X_pca_lbp.shape)

(5440, 1744)


RGB Features extractor

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# load the image data
img_folder_path = '/content/drive/MyDrive/Colab Notebooks/main_data/'
img_paths = []
labels = []
for i in range(10):
    folder_path = img_folder_path + str(i)
    for img_path in os.listdir(folder_path):
        img_paths.append(os.path.join(folder_path, img_path))
        labels.append(i)

# convert the images to RGB format and flatten them
X = []
for img_path in img_paths:
    img = cv2.imread(img_path)
    if img.shape[2] == 1: # convert grayscale to RGB
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    X.append(img.flatten())
X = np.array(X)

In [ ]:
# import PCA from sklearn
from sklearn.decomposition import PCA

# create PCA object with 95% variation
pca = PCA(n_components=0.95)

# fit and transform the flattened images
X_pca_rgb = pca.fit_transform(X)

In [ ]:
print(X_pca_rgb.shape)

(5440, 436)


CLBP feature extractor

In [ ]:
import numpy as np
import cv2
import os
import random
from google.colab import drive
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# Define path to image dataset folder
data_folder = '/content/drive/MyDrive/Colab Notebooks/main_data/'

# Define CLBP parameters
radius = 3
n_points = 8 * radius
METHOD = 'uniform'

# Create empty list to hold CLBP features and labels
lbp_features = []
labels = []

# Loop through image dataset folder and extract CLBP features and labels
for digit_folder in os.listdir(data_folder):
    digit_path = os.path.join(data_folder, digit_folder)
    if os.path.isdir(digit_path):
        for img_file in os.listdir(digit_path):
            img_path = os.path.join(digit_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            lbp = local_binary_pattern(img, n_points, radius, METHOD)
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-7)
            lbp_features.append(hist)
            labels.append(int(digit_folder))

# Convert CLBP features and labels to numpy arrays
lbp_features = np.array(lbp_features)
labels = np.array(labels)

In [ ]:
# import PCA from sklearn
from sklearn.decomposition import PCA

# create PCA object with 95% variation
pca = PCA(n_components=0.95)

# fit and transform the flattened images
X_pca_clbp = pca.fit_transform(lbp_features)
print(X_pca_clbp.shape)

(5440, 2)


In [ ]:
arr4 = np.hstack((X_pca_lbp,X_pca_rgb,X_pca_clbp))
print(arr4.shape)

(5440, 2182)


In [ ]:
import pandas as pd
label=pd.DataFrame(labels)
# Rename the column containing labels to "label"
label = label.rename(columns={0: "labels"})

# Verify the new column name
label.head()

,labels
0,8
1,8
2,8
3,8
4,8


In [ ]:
int_data = pd.DataFrame(arr4)
int_data.head()

,0,1,2,3,4,5,6,7,8,9,...,2172,2173,2174,2175,2176,2177,2178,2179,2180,2181
0,50.419765,-179.784635,988.403927,516.207354,308.215026,-619.781016,189.227200,-407.874345,126.754658,17.343743,...,70.341502,255.476858,12.104634,163.843635,-201.118995,59.543579,-100.526287,-107.715324,-0.028939,0.011341
1,-38.341512,153.004862,543.731125,623.855297,-48.968351,-10.645306,137.695185,259.033505,-238.826937,-527.094613,...,277.343393,169.065667,-43.881880,-24.864249,190.814048,-355.439417,-324.827785,156.287005,-0.016126,-0.001379
2,-95.775879,-169.407294,853.400650,283.021641,-125.142175,-378.697882,407.561901,-83.463059,27.907756,-149.767515,...,242.301629,36.130954,-61.709013,-69.628663,-77.538983,-239.099892,125.606931,9.599382,-0.028231,0.005881
3,112.686760,-22.338091,914.076188,516.554458,423.180620,-701.517922,153.735327,-184.058513,293.371816,-123.820895,...,321.864304,-14.546405,-262.870816,-107.872498,-205.675405,22.233615,-19.135115,113.431463,-0.004214,0.007148
4,-107.358225,-508.028544,-32.272708,588.073012,-179.129616,446.713652,-114.985539,-292.582267,43.653482,597.823915,...,-12.651140,6.113766,1.994951,52.627573,73.801168,-108.436218,103.413386,77.830666,-0.015065,0.005640


In [ ]:
int_data=pd.concat([int_data,label],axis=1)
int_data.head()

,0,1,2,3,4,5,6,7,8,9,...,2173,2174,2175,2176,2177,2178,2179,2180,2181,labels
0,50.419765,-179.784635,988.403927,516.207354,308.215026,-619.781016,189.227200,-407.874345,126.754658,17.343743,...,255.476858,12.104634,163.843635,-201.118995,59.543579,-100.526287,-107.715324,-0.028939,0.011341,8
1,-38.341512,153.004862,543.731125,623.855297,-48.968351,-10.645306,137.695185,259.033505,-238.826937,-527.094613,...,169.065667,-43.881880,-24.864249,190.814048,-355.439417,-324.827785,156.287005,-0.016126,-0.001379,8
2,-95.775879,-169.407294,853.400650,283.021641,-125.142175,-378.697882,407.561901,-83.463059,27.907756,-149.767515,...,36.130954,-61.709013,-69.628663,-77.538983,-239.099892,125.606931,9.599382,-0.028231,0.005881,8
3,112.686760,-22.338091,914.076188,516.554458,423.180620,-701.517922,153.735327,-184.058513,293.371816,-123.820895,...,-14.546405,-262.870816,-107.872498,-205.675405,22.233615,-19.135115,113.431463,-0.004214,0.007148,8
4,-107.358225,-508.028544,-32.272708,588.073012,-179.129616,446.713652,-114.985539,-292.582267,43.653482,597.823915,...,6.113766,1.994951,52.627573,73.801168,-108.436218,103.413386,77.830666,-0.015065,0.005640,8


In [ ]:
# Separate the feature matrix X and labels y
X = int_data.drop('labels', axis=1)
y = int_data['labels']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# create a SVM classifier
svm_clf = SVC(kernel='linear', probability=True, random_state=42)

# train the classifier on the training data
svm_clf.fit(X_train, y_train)

# predict the labels for the test data
y_pred_svm = svm_clf.predict(X_test)

# generate a classification report and print the evaluation metrics
print("SVM Classifier Results:\n")
#print(classification_report(y_test, y_pred_svm))
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Precision:", precision_score(y_test, y_pred_svm, average='macro'))
print("Recall:", recall_score(y_test, y_pred_svm, average='macro'))
print("F1 Score:", f1_score(y_test, y_pred_svm, average='macro'))
print("ROC AUC Score:", roc_auc_score(y_test, svm_clf.predict_proba(X_test), multi_class='ovr'))

SVM Classifier Results:

Accuracy: 0.6372549019607843
Precision: 0.6456587213657244
Recall: 0.6379923315525221
F1 Score: 0.6358492673481003
ROC AUC Score: 0.8984051008733678


In [ ]:
from xgboost import XGBClassifier

# Define XGBoost classifier
clf = XGBClassifier(n_estimators=100, max_depth=6, objective='multi:softmax', num_class=10)

# Train the classifier
clf.fit(X_train, y_train)

# Test the classifier and print classification report
y_pred = clf.predict(X_test)
#print(classification_report(y_test, y_pred_svm))
acc=accuracy_score(y_test, y_pred)
precision=precision_score(y_test, y_pred, average='macro')
recall=recall_score(y_test, y_pred, average='macro')
f1=f1_score(y_test, y_pred, average='macro')
roc_auc_xg=roc_auc_score(y_test, clf.predict_proba(X_test), multi_class='ovr')


print("Accuracy (XGBoost):", acc)
print("Precision (XGBoost):", precision)
print("Recall (XGBoost):", recall)
print("F1 Score (XGBoost):", f1)
print("ROC AUC score (XGBoost):", roc_auc_xg)

Accuracy (XGBoost): 0.7567401960784313
Precision (XGBoost): 0.754619411260131
Recall (XGBoost): 0.7541953063990021
F1 Score (XGBoost): 0.7528998833020428
ROC AUC score (XGBoost): 0.9640803080968692


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier on the training data
rfc.fit(X_train, y_train)

# Make predictions on the test data
rfc_pred = rfc.predict(X_test)

# Get predicted probabilities of the model
rfc_prob = rfc.predict_proba(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, rfc_pred)
precision = precision_score(y_test, rfc_pred, average='weighted')
recall = recall_score(y_test, rfc_pred, average='weighted')
f1 = f1_score(y_test, rfc_pred, average='weighted')
auc_roc = roc_auc_score(y_test, rfc_prob, multi_class='ovr')

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("AUC ROC Score:", auc_roc)

Accuracy: 0.6372549019607843
Precision: 0.6523267886430111
Recall: 0.6372549019607843
F1 Score: 0.6382485583819698
AUC ROC Score: 0.9073544983115204
